In [ ]:
import cPickle as pickle
import numpy as np

with open('gvars_g1.pkl', 'rb') as f:
    g1_vars = pickle.load(f)
    
with open('gvars_g2.pkl', 'rb') as f:
    g2_vars = pickle.load(f)
    
with open('snapshot_z.pkl', 'rb') as f:
    snapshot_z = pickle.load(f)
    
print snapshot_z.shape

In [ ]:
g1n_to_var = {}
g2n_to_var = {}
for (g1n, g1v), (g2n, g2v) in zip(g1_vars, g2_vars):
    g1n_to_var[g1n] = g1v
    g2n_to_var[g2n] = g2v
    print g1n, g1v.shape

In [ ]:
import tensorflow as tf

n2v = g2n_to_var

def pixel_norm(x):
    # NOTE: axis=-1 because we're using NWC rather than NCW
    return x / tf.sqrt(tf.reduce_mean(tf.square(x), axis=-1, keepdims=True) + 1e-8)

def wscale(x, scale, bias_fn, nonlinearity):
    return nonlinearity((x * scale) + n2v[bias_fn])

def upscale(x, scale=4):
    _, w, nch = x.get_shape().as_list()
    
    x = tf.expand_dims(x, axis=1)
    x = tf.image.resize_nearest_neighbor(x, [1, w * scale])
    x = x[:, 0]
    
    return x

conv_filter = lambda fn: np.transpose(n2v[fn], [2, 1, 0])[::-1]

tf.reset_default_graph()

z = tf.placeholder(tf.float32, [None, 512])

z_norm = pixel_norm(z)

x = tf.expand_dims(z_norm, axis=1)

# Conv1 (projects latents)
x = tf.pad(x, [[0, 0], [15, 15], [0, 0]])
x = tf.nn.conv1d(x, conv_filter('G1a.W'), 1, padding='VALID', data_format='NWC')
x = wscale(x, 0.015617936, 'G1aS.b', tf.nn.leaky_relu)
x = tf.nn.conv1d(x, conv_filter('G1b.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020833442, 'G1bS.b', tf.nn.leaky_relu)

# Conv2
x = upscale(x)
x = tf.nn.conv1d(x, conv_filter('G2a.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020837622, 'G2aS.b', tf.nn.leaky_relu)
x = tf.nn.conv1d(x, conv_filter('G2b.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020843236, 'G2bS.b', tf.nn.leaky_relu)

# Conv3
x = upscale(x)
x = tf.nn.conv1d(x, conv_filter('G3a.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020820292, 'G3aS.b', tf.nn.leaky_relu)
x = tf.nn.conv1d(x, conv_filter('G3b.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020836806, 'G3bS.b', tf.nn.leaky_relu)

# Conv4
x = upscale(x)
x = tf.nn.conv1d(x, conv_filter('G4a.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.02082933, 'G4aS.b', tf.nn.leaky_relu)
x = tf.nn.conv1d(x, conv_filter('G4b.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020813614, 'G4bS.b', tf.nn.leaky_relu)

# Conv5
x = upscale(x)
x = tf.nn.conv1d(x, conv_filter('G5a.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.020801041, 'G5aS.b', tf.nn.leaky_relu)
x = tf.nn.conv1d(x, conv_filter('G5b.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.0294875, 'G5bS.b', tf.nn.leaky_relu)

# Conv6
x = upscale(x)
x = tf.nn.conv1d(x, conv_filter('G6a.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.029462723, 'G6aS.b', tf.nn.leaky_relu)
x = tf.nn.conv1d(x, conv_filter('G6b.W'), 1, padding='SAME', data_format='NWC')
x = wscale(x, 0.04161643, 'G6bS.b', tf.nn.leaky_relu)

# Aggregate
f = np.reshape(g1n_to_var['Glod0.W'], [1, 128, 1])
x = tf.nn.conv1d(x, f, 1, padding='VALID', data_format='NWC')
Gz = wscale(x, 0.08200226, 'Glod0S.b', tf.identity)

In [ ]:
from IPython.display import display, Audio

with tf.Session() as sess:
    _Gz = sess.run(Gz, {z: snapshot_z})
    print _Gz[0, :, 0]
    display(Audio(_Gz[0, :, 0], rate=16000))

In [ ]:
from scipy.io.wavfile import read as wavread

_, ref = wavread('fakes008400.wav')

def adjust_dynamic_range(data, drange_in, drange_out):
    if drange_in != drange_out:
        scale = (np.float32(drange_out[1]) - np.float32(drange_out[0])) / (np.float32(drange_in[1]) - np.float32(drange_in[0]))
        bias = (np.float32(drange_out[0]) - np.float32(drange_in[0]) * scale)
        data = data * scale + bias
    return data

ref = np.reshape(ref, [8, -1])
ref = ref[:, :16384]
ref = ref.astype(np.float32)
ref = adjust_dynamic_range(ref, [-32768, 32767], [-1, 1])
print ref[0]
display(Audio(ref[0], rate=16000))